# Example notebook on how to make questions

The SAS token provides access cloud storage. Ask Floris Calkoen if you need one. 

In [ ]:
import json
import os
import pathlib

import dotenv
import fsspec
import panel as pn

import coastal_dynamics as cd

dotenv.load_dotenv(override=True)
account_name = "coclico"
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": account_name, "sas_token": sas_token}

## Make a database of questions

Make your questions in JSON format, following the example.json that is listed below - watch out for trailing comma's.
Also please follow the naming convention (keys) as in that example. The answers will be stored in a private cloud
container. With scripts/python/hash_questions.py we can hash the questions and store that in a public cloud container

In [ ]:
# Activate the Panel widget to make the visualizations.
pn.extension()


# Read both the unhashed and hashed answers
blob_name_non_hashed_answers = "az://coastal-dynamics/questions/example.json"
blob_name = "az://coastal-dynamics/questions/example_hashed.json"

questions = cd.read_questions(blob_name, storage_options)

# The QuestionFactory can handle the four question types.
q1 = cd.QuestionFactory(
    questions["Q1-1"]
).serve()  # Here 1-1 refers to notebook 1 question 1.
q2 = cd.QuestionFactory(questions["Q1-2"]).serve()
pn.Column(q1, q2)

## Additional examples for all four question types


In [ ]:
import pathlib
import warnings

import geoviews.tile_sources as gvts
import hvplot.pandas
import hvplot.xarray
import pooch
import requests

from coastal_dynamics.geometries import geo_bbox
from coastal_dynamics.visualization import DynamicWavePlot

pn.extension()

#### Multiple Choice

In [ ]:
cd.MultipleChoiceQuestion(
    question_name="Q1: Simple Multiple Choice Question",
    question_text="What coastal landform do you typically find in tide-dominated environments?",
    question_options={
        "a": "Mudflats",
        "b": "Open coasts",
        "c": "Cliffed coasts",
        "d": "Mixed sand and gravel beaches",
    },
    question_answer=cd.hash_answer("a", "multiple_choice"),
    question_feedback={
        "correct": "Indeed, ...",
        "incorrect": "Unfortunately incorrect. Please consder...",
    },
).serve()

#### Multiple Selection

In [ ]:
cd.MultipleSelectionQuestion(
    question_name="Q1: Simple Multiple Selection Question",
    question_text="Select all morphological systems that are commonly in coastal areas.",
    question_options={
        "a": "Beaches",
        "b": "Glaciers",
        "c": "Estuaries",
        "d": "Mountains",
        "e": "Sand",
        "f": "Mud",
    },
    question_answers=cd.hash_answer(["a", "c"], "multiple_selection"),
    question_feedback={
        "correct": "Indeed, ...",
        "incorrect": "Unfortunately incorrect. Please consder...",
    },
).serve()

#### Numeric question

In [ ]:
kwargs = {"precision": 2}
cd.NumericQuestion(
    question_name="Q3: Simple Numeric Question",
    question_text="What is the PSU of the global ocean?",
    question_answer=cd.hash_answer(35, "numeric", precision=1),
    question_feedback={
        "correct": "Indeed, ...",
        "incorrect": "Unfortunately incorrect. Please consder...",
    },
    precision=1,
).serve()

#### Text Question

In [ ]:
cd.TextQuestion(
    question_name="Q4: Some simple text question",
    question_text="How do you call the force that generates the tide?",
    question_answer=cd.hash_answer("Gravitational pull", "text"),
    question_feedback={
        "correct": "Indeed, ...",
        "incorrect": "Unfortunately incorrect. Please consder...",
    },
).serve().show()

### Some basis visualization blocks

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning, module="holoviews.core.data")
wave_plot = DynamicWavePlot(amplitude_range=(1, 3, 0.5), wavelength_range=(5, 20, 0.1))
wave_plot

In [ ]:
base_plot = gvts.EsriImagery(width=900, height=500)
base_plot

### Combine using panel

In [ ]:
pn.Column(wave_plot, base_plot)

### Combine all in a dashboard - works also for other questions

In [ ]:
all_questions_panel = pn.Column(sizing_mode="stretch_width")

# Add each question's panel to the main Panel column
for key, question_data in questions.items():
    question_panel = cd.QuestionFactory(question_data).serve()
    all_questions_panel.append(question_panel)

# Create a wave plot and a base plot
wave_plot = DynamicWavePlot(amplitude_range=(1, 3, 0.5), wavelength_range=(5, 20, 0.1))
base_plot = gvts.EsriImagery(width=900, height=500)

# Combine all components into a single dashboard
dashboard = pn.Column(
    pn.pane.Markdown("# Questions"),
    all_questions_panel,
    pn.pane.Markdown("# Coastal Dynamics Interactive Dashboard"),
    base_plot,
    wave_plot.__panel__(),  # Assuming DynamicWavePlot has a __panel__ method
    sizing_mode="stretch_width",
)
# or dashboard.show() for the browser
dashboard